In [1]:
from datasets import load_dataset

In [3]:
emotions = load_dataset('emotion')

Using custom data configuration default
Reusing dataset emotion (/home/sadegh/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train_ds = emotions['train']

## Character Tokenization

In [6]:
text = "Tokenizing text is a core task of NLP."
tokenized_text = list(text)

In [10]:
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}

In [25]:
import torch
import torch.nn.functional as F

In [29]:
input_ids = [token2idx[token] for token in tokenized_text]
input_ids = torch.tensor(input_ids)
one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx))

## Word Tokenization

In [31]:
tokenized_text = text.split()
tokenized_text

['Tokenizing', 'text', 'is', 'a', 'core', 'task', 'of', 'NLP.']

In [35]:
token2idx = {word: idx for idx, word in enumerate(sorted(set(tokenized_text)))}
input_ids = [token2idx[token] for token in token2idx]
input_ids = torch.tensor(input_ids)
one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx))
one_hot_encodings.shape

torch.Size([8, 8])

## Subword Tokenization

In [36]:
from transformers import AutoTokenizer

In [38]:
model_ckpt = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [39]:
from transformers import DistilBertTokenizer

In [40]:
tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)

In [42]:
encoded_text = tokenizer(text)
encoded_text

{'input_ids': [101, 19204, 6026, 3793, 2003, 1037, 4563, 4708, 1997, 17953, 2361, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [45]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)

In [46]:
tokenizer.convert_tokens_to_string(tokens)

'[CLS] tokenizing text is a core task of nlp . [SEP]'

## Tokenizing the Whole Dataset

In [49]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)